
## Цели работы

- Попрактиковаться в предобработке текстовых данных (токенизация, лемматизация, векторизация).

- Построить две модели классификации с разными способами векторизации текстов (`CountVectorizer` и `TfIdfVectorizer`).
- Сравнить качество моделей.

## Что нужно сделать

1. Загрузите датасет из файла, приложенного к заданию (`m18_jokes_dataset.csv`).

1. Закодируйте целевую переменную с помощью `LabelEncoder`.
1. Разделите выборку на тренировочную и тестовую.
1. Используя знания из модуля, подготовьте функцию `preprocess`, которая:
- удаляет стоп-слова в соответствии со списком `nltk.corpus.stopwords`;
- приводит слова к нормальной форме с помощью `pymorphy2`.
5. Создайте экзепляры векторизаторов `CountVectorizer` и `TfIdfVectorizer`. В качестве функции предобработки данных оба должны использовать функцию `preprocess`.

1. Векторизуйте признаки тренировочной и тестовой выборок. Сохраните результаты `CountVectorizer` в `X_train_count` и `X_test_count`, а результаты `TfIdfVectorizer` – в `X_train_tfidf` и `X_test_tfidf`.
1. Обучите две модели классификации, например `LinearSVC`.
1. Оцените работу обеих моделей на тестовой выборке, используя `f1_score` с методом усреднения `macro` (параметр `average='macro'`) – это арифметическое среднее f1-метрик, рассчитанных для каждого класса.

## Информация о задаче

### Описание датасета

`m18_jokes_dataset.csv` – датасет, подготовленный на основе датасета, размещённого [на kaggle](https://www.kaggle.com/datasets/konstantinalbul/russian-jokes). Наш датасет содержит две колонки:

- **text** — текст анекдота (скорее всего, несмешного, но ваша задача не смеяться, а решить задачу классификации);

- **theme** — категория, к которой относится шутка.

Датасет содержит данные для задачи многоклассовой классификации. В отличие от оригинального датасета, в нашем классов меньше, зато они идеально сбалансированы: в каждом ровно тысяча примеров. Так вам будет проще сконцентрироваться на текстовых данных.

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Импорт нужных библиотек

In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pymorphy2

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

### Загрузка данных

In [3]:
data = pd.read_csv(f'data/jokes_dataset.csv')
data.head()

,theme,text
0,aforizmi,"Почему удар в спину наносят те, кого,как прави..."
1,aforizmi,Музы - пожалуй самые мудрые представительницы...
2,aforizmi,"Тяжело бухать всю ночь, особенно если ночь пол..."
3,aforizmi,ПОКУШЕНИЕ. НАЛИЧНОСТЬ.\r\n\r\n\r\n
4,aforizmi,"Когда медленно танцуешь, ничего не мешает...\r..."


In [4]:
print('Соотношение классов в процентах:')
data.theme.value_counts()/len(data)*100

Соотношение классов в процентах:


aforizmi                 5.882353
pro-mugchin              5.882353
shkolnie-i-pro-shkolu    5.882353
raznie                   5.882353
pro-vovochku             5.882353
pro-studentov            5.882353
pro-semyu                5.882353
pro-novih-russkih        5.882353
pro-militsiyu            5.882353
meditsinskie             5.882353
pro-evreev               5.882353
pro-detey                5.882353
pro-armiu                5.882353
pro-alkogolikov          5.882353
poshlie-i-intimnie       5.882353
narodnie                 5.882353
tsitati                  5.882353
Name: theme, dtype: float64

### Подготовка данных для обучения

In [5]:
le = LabelEncoder()
data['theme'] = le.fit_transform(data['theme'])

In [6]:
X = data.text
y = data.theme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
def preprocess(text):
    lemm_words = []
    tokenizer = RegexpTokenizer('\w+')
    stop_words = stopwords.words('russian')
    morph = pymorphy2.MorphAnalyzer()
    for word in tokenizer.tokenize(text):
        word = word.lower()
        if word not in stop_words:
            lemm_words.append(morph.parse(word)[0].normal_form)
    return ' '.join(lemm_words)

In [8]:
count_vectorizer = CountVectorizer(preprocessor=preprocess)
count_vectorizer.fit(X_train)
X_train_count = count_vectorizer.transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

In [9]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocess)
tfidf_vectorizer.fit(X_train)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

### Создание и обучение моеделей LinearSVC

In [10]:
model_count = LinearSVC(max_iter=10000)
model_count.fit(X_train_count, y_train)
y_pred_count = model_count.predict(X_test_count)
score_count = f1_score(y_test, y_pred_count, average='macro')

model_tfidf = LinearSVC(max_iter=10000)
model_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
score_tfidf = f1_score(y_test, y_pred_tfidf, average='macro')

### Вывод метрик

In [11]:
print(f'f1_score при использовании Count_Vectorizer: {score_count}')
print(f'f1_score при использовании Tfidf_Vectorizer: {score_tfidf}')

f1_score при использовании Count_Vectorizer: 0.5996109085990549
f1_score при использовании Tfidf_Vectorizer: 0.6275222883803963
